In [118]:
stop_ids = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
# demand = {'A': 130, 'B': 48, 'C': 118, 'D': 115, 'E': 90, 'F': 62, 'G': 68}
stops = {'A': {'id': 'A', 'weight': 130, 'dist_to_wh': 632,
               'dist_to_stops': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374}},
         'B': {'id': 'B', 'weight': 48, 'dist_to_wh': 1081,
               'dist_to_stops': {'A': 661, 'C': 718, 'D': 1663, 'E': 1952, 'F': 1638, 'G': 1027}},
         'C': {'id': 'C', 'weight': 118, 'dist_to_wh': 628,
               'dist_to_stops': {'A': 756, 'B': 718, 'D': 982, 'E': 1422, 'F': 1331, 'G': 926}},
         'D': {'id': 'D', 'weight': 115, 'dist_to_wh': 741,
               'dist_to_stops': {'A': 1361, 'B': 1663, 'C': 982, 'E': 703, 'F': 1088, 'G': 1219}},
         'E': {'id': 'E', 'weight': 90, 'dist_to_wh': 872,
               'dist_to_stops': {'A': 1433, 'B': 1952, 'C': 1422, 'D': 703, 'F': 642, 'G': 1129}},
         'F': {'id': 'F', 'weight': 62, 'dist_to_wh': 707,
               'dist_to_stops': {'A': 1008, 'B': 1638, 'C': 1331, 'D': 1088, 'E': 642, 'G': 639}},
         'G': {'id': 'G', 'weight': 68, 'dist_to_wh': 499,
               'dist_to_stops': {'A': 374, 'B': 1027, 'C': 926, 'D': 1219, 'E': 1129, 'F': 639}}}
van_cap = 300

distances = {'A': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374},
             'B': {'C': 718, 'D': 1663, 'E': 1952, 'F': 1638, 'G': 1027},
             'C': {'D': 982, 'E': 1422, 'F': 1331, 'G': 926},
             'D': {'E': 703, 'F': 1088, 'G': 1219},
             'E': {'F': 642, 'G': 1129},
             'F': {'G': 639}}

distances_dc = {'A': 632, 'B': 1081, 'C': 628, 'D': 741, 'E': 872, 'F': 707, 'G': 499}

In [119]:
from collections import deque

routes_list = []
routes_dict = {}

class Route(object):

    max_load = van_cap

    def __init__(self, stop, measure):

        assert stop[measure] <= Route.max_load, "exceeds max_load_1"
        self.llist = deque([stop])
        self.measure = measure  # string used as key to dict
        self.load = stop[measure]
        self.spare_cap = Route.max_load - self.load

        self.first_stop = stop['id']
        self.last_stop = stop['id']
        self.first_and_last_stops = (self.first_stop, self.last_stop)

        self.n_stops = 1
        self.total_dist = stop['dist_to_wh'] * 2

    def merge(self, route, saving):
        if self.n_stops > 1:
            keep = self
            drop = route
        else:
            keep = route
            drop = self

        keep.update(drop, saving)
        drop.eliminate()

    def update(self, route, saving):
        if self.first_stop in saving.nodes:
            self.new_first_stop(route.llist[0])
        elif self.last_stop in saving.nodes:
            self.new_last_stop(route.llist[0])

    def eliminate(self):
        routes_list.remove(self)
        routes_dict[self.first_stop].remove(self)
        if self.n_stops > 1:
            routes_dict[self.last_stop].remove(self)

    def new_first_stop(self, stop):

        assert stop[self.measure] <= self.spare_cap, "exceeds max_load_2"

        self.total_dist += stop['dist_to_wh'] - self.llist[0]['dist_to_wh'] \
                           + stop['dist_to_stops'][self.first_stop]

        self.llist.appendleft(stop)

        if self.n_stops > 1:
            routes_dict[self.first_stop].remove(self)
        self.first_stop = stop['id']
        routes_dict[self.first_stop].append(self)

        self.first_and_last_stops = (self.first_stop, self.last_stop)
        self.n_stops += 1
        self.load += stop[self.measure]
        self.spare_cap = Route.max_load - self.load



    def new_last_stop(self, stop):

        assert stop[self.measure] <= self.spare_cap, "exceeds max_load_3"

        self.total_dist += stop['dist_to_wh'] - self.llist[-1]['dist_to_wh'] \
                           + stop['dist_to_stops'][self.last_stop]

        self.llist.append(stop)

        if self.n_stops > 1:
            routes_dict[self.last_stop].remove(self)
        self.last_stop = stop['id']
        routes_dict[self.last_stop].append(self)

        self.first_and_last_stops = (self.first_stop, self.last_stop)
        self.n_stops += 1
        self.load += stop[self.measure]
        self.spare_cap = Route.max_load - self.load


# build the list of initial routes:
routes_list = [Route(stops[each], measure='weight') for each in stops]

# index routes_list in a dict, by the stop_id of first or last:

for each in stops:
    routes_dict[each] = []

for route in routes_list:
    for each in route.first_and_last_stops:
        if route not in routes_dict[each]:
            routes_dict[each].append(route)

# this needs to be updated incrementally after each merge

In [120]:
# total distance if round trips with one stop

def base_total_cost(adict):
    sum = 0
    for key in adict:
        sum += 2 * adict[key]
    return sum

base_tc = base_total_cost(distances_dc)

In [121]:
class Saving(object):

    def __init__(self, start, end, saving):
        self.nodes = (start, end)
        self.saving = saving
        self.used = None  # will be modified by algorithm

    def __lt__(self, other):
        return self.saving <= other.saving

    def __str__(self):
        return str(self.nodes) + ", " + str(self.saving)

savings = [Saving(key, nested_key,
                  distances_dc[key] + distances_dc[nested_key] - distances[key][nested_key])
           for key in distances for nested_key in distances[key]]

def sort(a_list, asc=True):
    """takes in a list of objects, returns it sorted"""
    for i in range(len(a_list)):
        for j in range(len(a_list)):
            if asc and a_list[j] > a_list[i]:
                a_list[j], a_list[i] = a_list[i], a_list[j]
            if not asc and a_list[j] < a_list[i]:
                a_list[j], a_list[i] = a_list[i], a_list[j]

    return a_list

# create a dict of savings, indexed by saving.used - the 'None' are sorted DESC:
# savings = {'None': sort(savings, asc=False), 'True': [], 'False': []}

In [122]:
# https://realpython.com/sorting-algorithms-python/

from random import randint

def quicksort(array, asc=False):

    if len(array) < 2:
        return array

    low, same, high = [], [], []

    pivot = array[randint(0, len(array) - 1)]

    for item in array:
        if item < pivot:
            low.append(item)
        elif item == pivot:
            same.append(item)
        elif item > pivot:
            high.append(item)

    if asc:
        return quicksort(low, asc) + same + quicksort(high, asc)
    if not asc:
        return quicksort(high, asc) + same + quicksort(low, asc)

# create a dict of savings, indexed by saving.used - the 'None' are sorted DESC:
savings = {'None': quicksort(savings, asc=False), 'True': [], 'False': []}

In [123]:
# for saving_candidate in savings['None']:
#     # 1- select TOP saving from ordered list of candidates:
#
#     # 2- select routes that start or end with one of the candidate's nodes - must be strictly less than 3:
#     routes_before = [each for each in routes_dict[saving_candidate.nodes[0]]] # search for node[0]
#     routes_before.extend([each for each in routes_dict[saving_candidate.nodes[1]]])  # search for node[1]
#     assert len(routes_before) < 3, "too many routes, check selection"
#
#     # 3- if exactly 2 routes and combined load less than max:
#     if len(routes_before) == 2:
#         if routes_before[0].load + routes_before[1].load <= routes_before[0].max_load:
#             routes_before[0].merge(routes_before[1], saving_candidate)
#             saving_candidate.used = True
#             # savings['None'].remove(saving_candidate)
#             savings['True'].append(saving_candidate)
#         else:
#             saving_candidate.used = False
#             savings['False'].append(saving_candidate)
#     else:
#             saving_candidate.used = False
#             savings['False'].append(saving_candidate)

In [124]:
print("hello")

hello
